In [2]:
import pandas as pd
import numpy as np
import re
import ast

In [3]:
spy_stock_data = pd.read_csv('../data/processed/SPY.csv')
spy_news_data = pd.read_csv('../data/processed/SPY.US_news.csv')

print("SPY Stock Data:")
print(spy_stock_data.head())

print("\nSPY News Data:")
print(spy_news_data.head())

SPY Stock Data:
         Date        Open        High         Low       Close  Adj_Close  \
0  2010-01-06  113.519997  113.989998  113.430000  113.709999  86.871063   
1  2010-01-07  113.500000  114.330002  113.180000  114.190002  87.237770   
2  2010-01-08  113.889999  114.620003  113.660004  114.570000  87.528069   
3  2010-01-11  115.080002  115.129997  114.239998  114.730003  87.650307   
4  2010-01-12  113.970001  114.209999  113.220001  113.660004  86.832848   

      Volume  Log_Return_Close_Close  Log_Return_Open_Close  \
0  116074400                0.000704               0.001672   
1  131091100                0.004212               0.006061   
2  126402800                0.003322               0.005953   
3  106375700                0.001396              -0.003046   
4  163333500               -0.009370              -0.002724   

   Log_Return_High_Low  Log_Return_Open_Open  Log_Return_Close_Open  \
0             0.004925              0.002293              -0.000969   
1     

In [4]:
# Preprocess and Clean the News Data
# ------------------------------------------
# The sentiment column is stored as JSON-like strings, we need to extract them.
# We'll parse the 'sentiment' column and clean it.

# Function to parse sentiment column (stored as JSON-like strings)
def parse_sentiment(sentiment_str):
    try:
        return ast.literal_eval(sentiment_str)  # Convert string to dictionary
    except (ValueError, SyntaxError):
        return None


# Apply the parsing function to the sentiment column
spy_news_data['parsed_sentiment'] = spy_news_data['sentiment'].apply(parse_sentiment)

# Now, extract 'polarity', 'neg', 'neu', and 'pos' from the parsed sentiment
spy_news_data['polarity'] = spy_news_data['parsed_sentiment'].apply(lambda x: x['polarity'] if x else None)
spy_news_data['neg'] = spy_news_data['parsed_sentiment'].apply(lambda x: x['neg'] if x else None)
spy_news_data['neu'] = spy_news_data['parsed_sentiment'].apply(lambda x: x['neu'] if x else None)
spy_news_data['pos'] = spy_news_data['parsed_sentiment'].apply(lambda x: x['pos'] if x else None)

# Drop the unnecessary 'parsed_sentiment' column
spy_news_data = spy_news_data.drop(columns=['parsed_sentiment'])

# Convert the 'date' column to just the date (without time)
spy_news_data['date_only'] = pd.to_datetime(spy_news_data['date']).dt.date

# Inspect the cleaned news data
print("\nCleaned SPY News Data:")
print(spy_news_data[['date_only', 'polarity', 'neg', 'neu', 'pos']].head())


Cleaned SPY News Data:
    date_only  polarity    neg    neu    pos
0  2024-09-10     0.960  0.052  0.861  0.087
1  2024-09-06    -0.840  0.088  0.847  0.065
2  2024-09-06    -0.773  0.106  0.801  0.094
3  2024-09-06     0.972  0.107  0.750  0.143
4  2024-09-05     0.992  0.084  0.784  0.132


In [5]:
# Aggregate the News Sentiment Scores by Date
# ---------------------------------------------------
# Since there may be multiple articles for the same day, we'll aggregate the sentiment scores.

aggregated_sentiment = spy_news_data.groupby('date_only').agg({
    'polarity': 'mean',
    'neg': 'mean',
    'neu': 'mean',
    'pos': 'mean'
}).reset_index()

# Inspect the aggregated sentiment data
print("\nAggregated Sentiment Data:")
print(aggregated_sentiment.head())


Aggregated Sentiment Data:
    date_only  polarity       neg       neu     pos
0  2022-04-04    0.0350  0.038600  0.901400  0.0602
1  2022-04-05    0.3865  0.017167  0.919833  0.0630
2  2022-04-06   -0.5880  0.065000  0.879000  0.0550
3  2022-04-07    0.6818  0.046800  0.873200  0.0800
4  2022-04-08    0.9546  0.043000  0.856000  0.1008


In [6]:
# Step 5: Prepare the Stock Data for Merging
# ------------------------------------------
# Ensure the 'Date' column in the stock data is in the correct format (YYYY-MM-DD) for merging.
spy_stock_data['Date'] = pd.to_datetime(spy_stock_data['Date']).dt.date

# Inspect the stock data
print("\nSPY Stock Data Ready for Merging:")
print(spy_stock_data.head())


SPY Stock Data Ready for Merging:
         Date        Open        High         Low       Close  Adj_Close  \
0  2010-01-06  113.519997  113.989998  113.430000  113.709999  86.871063   
1  2010-01-07  113.500000  114.330002  113.180000  114.190002  87.237770   
2  2010-01-08  113.889999  114.620003  113.660004  114.570000  87.528069   
3  2010-01-11  115.080002  115.129997  114.239998  114.730003  87.650307   
4  2010-01-12  113.970001  114.209999  113.220001  113.660004  86.832848   

      Volume  Log_Return_Close_Close  Log_Return_Open_Close  \
0  116074400                0.000704               0.001672   
1  131091100                0.004212               0.006061   
2  126402800                0.003322               0.005953   
3  106375700                0.001396              -0.003046   
4  163333500               -0.009370              -0.002724   

   Log_Return_High_Low  Log_Return_Open_Open  Log_Return_Close_Open  \
0             0.004925              0.002293              

In [7]:
# Merge Stock Data with Aggregated Sentiment Data
# -------------------------------------------------------
# Merge the stock data with the aggregated sentiment data on the 'Date' column.
combined_data = pd.merge(spy_stock_data, aggregated_sentiment, left_on='Date', right_on='date_only', how='left')

# Drop the redundant 'date_only' column
combined_data = combined_data.drop(columns=['date_only'])

# Inspect the combined data
print("\nCombined Stock and Sentiment Data:")
print(combined_data.head())



Combined Stock and Sentiment Data:
         Date        Open        High         Low       Close  Adj_Close  \
0  2010-01-06  113.519997  113.989998  113.430000  113.709999  86.871063   
1  2010-01-07  113.500000  114.330002  113.180000  114.190002  87.237770   
2  2010-01-08  113.889999  114.620003  113.660004  114.570000  87.528069   
3  2010-01-11  115.080002  115.129997  114.239998  114.730003  87.650307   
4  2010-01-12  113.970001  114.209999  113.220001  113.660004  86.832848   

      Volume  Log_Return_Close_Close  Log_Return_Open_Close  \
0  116074400                0.000704               0.001672   
1  131091100                0.004212               0.006061   
2  126402800                0.003322               0.005953   
3  106375700                0.001396              -0.003046   
4  163333500               -0.009370              -0.002724   

   Log_Return_High_Low  Log_Return_Open_Open  Log_Return_Close_Open  \
0             0.004925              0.002293             

In [8]:
# Handle Missing Data by Carrying Over the Previous Day's Sentiment
# ------------------------------------------------------------------------
# Use forward-fill (ffill) to carry over the previous day's sentiment for missing values

combined_data[['polarity', 'neg', 'neu', 'pos']] = combined_data[['polarity', 'neg', 'neu', 'pos']].ffill()

# Inspect the data after applying forward-fill
print("\nCombined Data After Forward-Filling Missing Values:")
print(combined_data.head())


Combined Data After Forward-Filling Missing Values:
         Date        Open        High         Low       Close  Adj_Close  \
0  2010-01-06  113.519997  113.989998  113.430000  113.709999  86.871063   
1  2010-01-07  113.500000  114.330002  113.180000  114.190002  87.237770   
2  2010-01-08  113.889999  114.620003  113.660004  114.570000  87.528069   
3  2010-01-11  115.080002  115.129997  114.239998  114.730003  87.650307   
4  2010-01-12  113.970001  114.209999  113.220001  113.660004  86.832848   

      Volume  Log_Return_Close_Close  Log_Return_Open_Close  \
0  116074400                0.000704               0.001672   
1  131091100                0.004212               0.006061   
2  126402800                0.003322               0.005953   
3  106375700                0.001396              -0.003046   
4  163333500               -0.009370              -0.002724   

   Log_Return_High_Low  Log_Return_Open_Open  Log_Return_Close_Open  \
0             0.004925              0.00

In [9]:
# Save the Combined Data
# ------------------------------
# Save the merged DataFrame to a new CSV file for further use.
combined_data.to_csv('../data/processed/combined_spy_stock_sentiment.csv', index=False)

print("\nData saved to 'combined_spy_stock_sentiment.csv'!")


Data saved to 'combined_spy_stock_sentiment.csv'!
